In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sp_spec

In [ ]:
def read_P_data():
    """
    Read fit of TF for E/I neurons
    needs to be in the same file as the fits for P
    """
    f = open('RS_fit.txt', 'r')
    lines = f.readlines()
    PRS=np.zeros(10)
    for i in range(0,len(PRS)):
        PRS[i]=lines[i]
    f.close()

In [ ]:
    f = open('FS_fit.txt', 'r')
    lines = f.readlines()
    PFS=np.zeros(10)
    for i in range(0,len(PRS)):
        PFS[i]=lines[i]
    f.close()
    
    return PRS, PFS

In [ ]:
def mu_v(nue, nui, u, muv_params):
    
    giz, Eiz, Qe, Qi, Ke, Ki, tause, tausi, Ee, Ei, b_adap = muv_params
    muge=Qe*nue*tause*Ke
    mugi=Qi*nui*tausi*Ki
    
    muv=((2*giz*Eiz + muge + mugi + b_adap) - \
          np.sqrt((2*giz*Eiz + muge + mugi + b_adap)**2 - 4*giz*(giz*Eiz**2 + muge*Ee + mugi*Ei -u)))/(2*giz)
    
    return muv

In [ ]:
def sv(ne, ni, ue, ui, sv_params, muve_params, muvi_params):
    #Standard deviation of the membrane potentials
    
    giz, Eiz, gizi, Eizi, Qe, Qi, Ee, Ei, tause, tausi, Ke, Ki, tauize, tauizi = sv_params
    muve = mu_v(ne, ni, ue, muve_params)
    muvi = mu_v(ne, ni, ui, muvi_params)
    ae=giz*(muve-Eiz)**2
    ai=gizi*(muvi-Eizi)**2
 
    be=Qe*(Ee-muve)
    bi=Qi*(Ei-muvi)
    ce=tause
    ci=tausi
    
    argsv=Ke*ne*(2*ae*be*ce**3/tauize**2+ce**3*be**2/(8*tauize)+be**2*ce**3/(8*tauize**2)) + Ki*ni*(2*ai*bi*ci**3/tauizi**2+ci**3*bi**2/(8*tauizi)   +bi**2*ci**3/(8*tauizi**2))
    if argsv>0:
        sv=np.sqrt(Ke*ne*(2*ae*be*ce**3/tauize**2+ce**3*be**2/(8*tauize)+be**2*ce**3/(8*tauize**2)) + Ki*ni*(2*ai*bi*ci**3/tauizi**2+ci**3*bi**2/(8*tauizi)+bi**2*ci**3/(8*tauizi**2)))
    else:
        sv=1e-9
        
    return sv
    
def tauv(ne, ni, ue, ui, tauv_params, muve_params, muvi_params):
    #characteristic time of the membrane potential evolution
    ne=ne+1e-9; ni=ni+1e-9 #avoid divide by 0's
    giz, Eiz, gizi, Eizi, Qe, Qi, Ee, Ei, tause, tausi, Ke, Ki, tauize, tauizi = tauv_params
    muve = mu_v(ne, ni, ue, muve_params)
    muvi = mu_v(ne, ni, ui, muvi_params)
    ae=giz*(muve-Eiz)**2
    ai=gizi*(muvi-Eizi)**2
    be=Qe*(Ee-muve)
    bi=Qi*(Ei-muvi)
    ce=tause
    ci=tausi
  
    
    argv=Ke*ne*(2*ae*be*ce**3/tauize**2+ce**3*be**2/(8*tauize)+be**2*ce**3/(8*tauize**2)) + Ki*ni*(2*ai*bi*ci**3/tauizi**2+ci**3*bi**2/(8*tauizi)+bi**2*ci**3/(8*tauizi**2))
    
    if argv >0:
        svv2=Ke*ne*(2*ae*be*ce**3/tauize**2+ce**3*be**2/(8*tauize)+be**2*ce**3/(8*tauize**2)) + Ki*ni*(2*ai*bi*ci**3/tauizi**2+ci**3*bi**2/(8*tauizi)+bi**2*ci**3/(8*tauizi**2))
    
        tauv=0.5*(Ke*ne*(be**2*ce**4/(2*np.pi*tauize**2))+Ki*ni*(bi**2*ci**4/(2*np.pi*tauizi**2)))/(svv2+1e-9)
        
    else:
      tauv=1
    return tauv

In [ ]:
def TF(X, nu, P, ue, ui, sv_params, muvi_params, muve_params):
    #transfer function
    #nu: mean membrane pot
    #ue: adaptation for the excitatory pop
    #P: terms of the polynom Veff thr
    #giz, Eiz: terms of the Izhikevich neuro,
    
    gize, Eize, gizi, Eizi, Qe, Qi, Ee, Ei, tause, tausi, Ke, Ki, tauize, tauizi = sv_params
    
    ne,ni=nu
    Po,Pmuv,Psv,Ptauv,Pvsv,Pvtauv,Psvtauv,Pvv,Ptt,Pss=P
    muvo=-45
    dmuvo=2
    svo=4
    dsvo=5
    tauvo=0.005
    dtauvo=0.05
 
    if X == "E":
        muv = mu_v(ne, ni, ue, muve_params)
        
    elif X == "I":
        muv = mu_v(ne, ni, ui, muvi_params)
    
    else:
        print("Must specify if excitatory or inhibitory")
    
    stdv = sv(ne, ni, ue, ui, sv_params, muve_params, muvi_params)
    tau_v = tauv(ne, ni, ue, ui, sv_params, muve_params, muvi_params)
 
    Pscale=1.
    
    noutf=Pscale*(sp_spec.erfc(((Po + Pmuv*(muv-muvo)/dmuvo + Psv*(stdv-svo)/dsvo + Ptauv*(tau_v-tauvo)/dtauvo \
            +Pvsv*(muv-muvo)*(stdv-svo)/(dsvo*dmuvo) + Pvtauv*(muv-muvo)*(tau_v-tauvo)/(dtauvo*dmuvo)\
            +Psvtauv*(stdv-svo)*(tau_v-tauvo)/(dtauvo*dsvo) + Pvv*(muv-muvo)**2/(dmuvo*dmuvo) \
            +Ptt*(tau_v-tauvo)**2/(dtauvo*dtauvo) + Pss*(stdv-svo)**2/(dsvo*dsvo))-muv)/(np.sqrt(2)*stdv) ))/(2*tau_v)
    return noutf

In [ ]:
def MF_run(nueIni, nuiIni, external_input, PRS, PFS, aRS, dRS, sv_params, muvi_params, muve_params, T, TotTime, timestep, percentage):
    t = np.linspace(0, TotTime, int(TotTime/timestep))
    
    fecont = nueIni; ficont = nuiIni
    w=0.5*fecont*dRS/aRS
    LSw=[]
    LSfe=[]
    LSfi=[]
    Pe = []
    Pi = []
    for i in range(len(t)):
        #TF(X, nu, P, ue, ui)
        nue = fecont; nui = ficont
        muve = mu_v(nue, nui, w, muve_params)
        muvi = mu_v(nue, nui, 0, muvi_params)
        
        fecontold=fecont
        fecont+=timestep/T*(TF("E", (fecont+external_input,ficont), PRS, w, 0, sv_params, muvi_params, muve_params)-fecont) 
        w+=timestep*( -aRS*w+(dRS)*fecontold)
        ficont+=timestep/T*(TF("I", (fecont+external_input,ficont),PFS, w ,0, sv_params, muvi_params, muve_params) - ficont) 
        
        
        LSfe.append(float(fecont))
        LSfi.append(float(ficont))
        LSw.append(float(w))
        Pe.append(float(muve))
        Pi.append(float(muvi))
        
    trajectories = [t, LSw, LSfe, LSfi, Pe, Pi]
    
    mean_firing_rate_E = np.mean(LSfe[int(percentage*len(LSfe))::])
    mean_firing_rate_I = np.mean(LSfi[int(percentage*len(LSfi))::])
    mean_pot_I = np.mean(Pi[int(percentage*len(Pi))::])
    mean_pot_E = np.mean(Pe[int(percentage*len(Pe))::])
    
    mean_results = [mean_pot_E, mean_pot_I, mean_firing_rate_E, mean_firing_rate_I]
    
    return trajectories, mean_results